In [89]:
import pandas as pd
import re
import numpy as np

# 显示所有列
pd.set_option('display.max_columns', None)

print("=== 开始执行数据处理流程 ===")
# =============================================================================
# 步骤
# =============================================================================
df = pd.read_csv('recruit-src.csv')
print(df.columns)

=== 开始执行数据处理流程 ===
Index(['城市', '关键词', '职位', '职位详情链接', '地区', '薪资范围', '经验要求', '学历要求', '公司名称',
       '公司详情链接', '公司类型', '融资情况', '公司规模', '标签1', '标签2', '标签3', '标签4', '标签5',
       '福利待遇', '创建时间'],
      dtype='object')


# 步骤1：初始数据清洗

In [ ]:
df = df.rename(columns={'关键词': '岗位'})
# 删除冗余链接列
df.drop(columns=['职位详情链接', '公司详情链接', '职位', '地区','福利待遇','标签1','标签2','标签3','标签4','标签5'], inplace=True)
# 数据去重
df.drop_duplicates(inplace=True)
# 添加自增ID列
df.insert(0, 'id', range(1, len(df) + 1))

# 保存中间结果
print(df.head().to_string())
print(f"✓ 当前数据：{len(df)} 条记录，{len(df.columns)} 列")

   id  城市      岗位        薪资范围   经验要求 学历要求              公司名称       公司类型    融资情况        公司规模                 创建时间
0   1  广州  Java后端       8-12K   3-5年   本科              震海集团        互联网   不需要融资    100-499人  2024-09-10 16:00:00
1   2  广州  Java后端        4-6K   1年以内   大专              半度文化      广播/影视     未融资      20-99人  2024-06-01 07:00:00
2   3  广州  Java后端       9-12K   3-5年   大专               葡萄柚        互联网  20-99人         NaN  2024-03-05 15:00:00
3   4  广州  Java后端  20-30K·14薪  5-10年   本科            老百姓大药房       医疗健康     已上市    10000人以上  2024-12-15 06:00:00
4   5  广州  Java后端      20-30K   3-5年   本科  某大型生活服务(O2O)上市公司  生活服务(O2O)     已上市  1000-9999人  2024-04-03 13:00:00
✓ 当前数据：13197 条记录，11 列


# 薪资范围处理

In [91]:
# 定义薪资提取函数
def extract_salary_midpoint(salary_str):
    match = re.match(r'(\d+)-(\d+)K(?:·(\d+)薪)?', salary_str)
    if match:
        lower, upper = int(match.group(1)), int(match.group(2))
        return (lower + upper) / 2
    return np.nan

In [92]:
# 应用薪资提取
df['薪资范围'] = df['薪资范围'].apply(extract_salary_midpoint)
df = df.rename(columns={'薪资范围': '薪资'})

# 清理无效数据
df = df.dropna(subset=['薪资'])

print(df[['薪资', '经验要求', '公司规模']].head().to_string())
print(f"✓ 当前数据：{len(df)} 条记录，{len(df.columns)} 列")

     薪资   经验要求        公司规模
0  10.0   3-5年    100-499人
1   5.0   1年以内      20-99人
2  10.5   3-5年         NaN
3  25.0  5-10年    10000人以上
4  25.0   3-5年  1000-9999人
✓ 当前数据：13061 条记录，11 列


### 经验

In [93]:
def extract_min_experience(experience_str):
    # 先过滤特殊情况
    if experience_str == '1年以内':
        return 0
    elif experience_str == '经验不限':
        return 0

    # 再尝试匹配格式如 "3-5年" 或 "2年"
    match = re.match(r'(\d+)-?(\d*)年', experience_str)
    if match:
        min_exp = int(match.group(1))
        return min_exp

    # 无法识别的情况返回 NaN
    return np.nan

In [94]:
# 应用函数并创建新列
df['经验要求'] = df['经验要求'].apply(extract_min_experience)
df = df.rename(columns={'经验要求': '年限'})

# 删除“最低经验要求”为空的行
df = df.dropna(subset=['年限'])

### 规模

In [95]:
# 定义 extract_scale 函数：从融资情况中提取公司规模
def extract_scale(financing_info):
    if not isinstance(financing_info, str):
        return None
    match = re.search(r'(\d+-\d+|\d+)人|(\d+)以上', financing_info)
    if match:
        return match.group(1) or match.group(2)
    return None

# 定义 calculate_midpoint 函数：计算公司规模的中值
def calculate_midpoint(scale):
    if not scale:
        return None
    scale = scale.replace('人', '')  # 去除“人”字
    if '-' in scale:
        lower, upper = map(int, scale.split('-'))
        return (lower + upper) // 2
    elif scale.isdigit():
        return int(scale)
    elif '以上' in scale:
        return int(scale.replace('以上', ''))
    else:
        return None

# 合并处理函数：用于 apply
def process_scale(row):
    scale = row['公司规模']
    if pd.isna(scale) or scale.strip() == '':
        scale = extract_scale(row['融资情况'])
    return calculate_midpoint(scale)

# 添加新列：公司规模中值
df['公司规模'] = df.apply(process_scale, axis=1)

# 删除原始列
df = df.drop(columns=['融资情况'])

In [96]:
print(df)

          id  城市      岗位    薪资    年限 学历要求              公司名称       公司类型   公司规模  \
0          1  广州  Java后端  10.0   3.0   本科              震海集团        互联网    299   
1          2  广州  Java后端   5.0   0.0   大专              半度文化      广播/影视     59   
2          3  广州  Java后端  10.5   3.0   大专               葡萄柚        互联网     59   
3          4  广州  Java后端  25.0   5.0   本科            老百姓大药房       医疗健康  10000   
4          5  广州  Java后端  25.0   3.0   本科  某大型生活服务(O2O)上市公司  生活服务(O2O)   5499   
...      ...  ..     ...   ...   ...  ...               ...        ...    ...   
13192  13193  重庆      产品  12.5   0.0   大专            重庆苏必利尔       电子商务     59   
13193  13194  重庆      产品  45.0  10.0   本科    北京某大型汽车研发/制造公司    汽车研发/制造   5499   
13194  13195  重庆      产品  14.5   0.0   本科              德勤勤跃       企业服务     59   
13195  13196  重庆      产品  12.5   3.0   本科               福玛特  智能硬件/消费电子    299   
13196  13197  重庆      产品  45.0  10.0   本科      某大型汽车研发/制造公司    汽车研发/制造   5499   

                      创建时间 

### 时间

In [97]:
df.to_csv('recruit.csv', index=False)